In [1]:
import pandas as pd
import os
from datetime import datetime

In [2]:
# Baca file CSV
file_path = 'asosiasi/results/rantau_dedap/gamma_picks.csv'
data = pd.read_csv(file_path)

In [3]:
# Pastikan kolom memiliki nama yang sesuai, berikut asumsi kolom:
# 'id', 'timestamp', 'prob', 'type', 'filename', 'event_index', 'gamma_score'
data.columns = ['id', 'timestamp', 'prob', 'type', 'filename', 'event_index', 'gamma_score']

In [4]:
data

id                   timestamp      prob type  \
0     RD13  2024-12-01T02:12:46.380000  0.117010    S   
1     RD13  2024-12-01T02:12:46.660000  0.239986    P   
2     RD14  2024-12-01T02:12:47.120000  0.166764    P   
3     RD14  2024-12-01T02:12:47.280000  0.170755    S   
4     RD15  2024-12-01T02:12:47.070000  0.301039    S   
...    ...                         ...       ...  ...   
1259  RD14  2024-12-31T22:47:24.920000  0.109618    P   
1260  RD11  2024-12-31T22:47:25.440000  0.131185    P   
1261  RD11  2024-12-31T22:47:25.490000  0.291042    S   
1262  RD13  2024-12-31T22:47:25.410000  0.153500    S   
1263  RD15  2024-12-31T22:47:25.120000  0.555975    P   

                            filename  event_index  gamma_score  
0     RD13_(B242)_RD_20241201_020000            1     0.027921  
1     RD13_(B242)_RD_20241201_020000            1     0.064634  
2     RD14_(B158)_RD_20241201_020000            1     0.042241  
3     RD14_(B158)_RD_20241201_020000            1     0.045630  
4     RD15_(AD18)_RD_20241201_020000            1     0.080529  
...                              ...          ...          ...  
1259  RD14_(B158)_RD_20241231_220000          139     0.031027  
1260  RD11_(B232)_RD_20241231_220000          139     0.038424  
1261  RD11_(B232)_RD_20241231_220000          139     0.085743  
1262  RD13_(B242)_RD_20241231_220000          139     0.044651  
1263  RD15_(AD18)_RD_20241231_220000          139     0.088010  

[1264 rows x 7 columns]

In [9]:
# Pisahkan data berdasarkan event_index
output_dir = "D:/SERD/01_meq/00_seisgram2k/picks_2024_12/"
os.makedirs(output_dir, exist_ok=True)

for event_index, group in data.groupby('event_index'):
    # Ambil tanggal dari timestamp untuk penamaan file
    first_timestamp = group['timestamp'].iloc[0]
    date_part = first_timestamp.split('T')[0]
    yyyy, mm, dd = date_part.split('-')
    julian_day = datetime.strptime(date_part, "%Y-%m-%d").timetuple().tm_yday

    # Format nama file: event(2 digit)_YYYYMMDD_julianday
    output_file = os.path.join(output_dir, f'{int(event_index):02d}_{yyyy}{mm}{dd}_{julian_day}.pick')

    with open(output_file, 'w') as f:
        for _, row in group.iterrows():
            station_id = row['id']
            phase = row['type'].upper()
            timestamp = row['timestamp']
            gamma_score = row['gamma_score']

            # Tentukan BHZ atau BHN
            bh_type = 'BHZ' if phase == 'P' else 'BHN'

            # Format waktu dan tanggal
            date_part, time_part = timestamp.split('T')
            yyyy, mm, dd = date_part.split('-')
            hh, mi, sec = time_part.split(':')
            sec = float(sec)  # Konversi detik ke format desimal

            # Pastikan jam tidak melebihi 23
            hh = int(hh) % 24

            # Format output sesuai spesifikasi
            line = f"{station_id} ? {bh_type} e {phase} c {yyyy}{mm}{dd} {hh:02d}{mi} {sec:06.3f} GAU 0.0 0.0 0.0 0.0\n"
            f.write(line)

print(f"File dipisahkan ke folder: {output_dir}")

File dipisahkan ke folder: D:/SERD/01_meq/00_seisgram2k/picks_2024_12/
